# __Ocean regimes indicator__: Model development notebook

Create a Gaussian Mixtures Model trained with a dataset of time series

***
ds : initial dataset (lat, lon, week)

X : staked dataset (sampling, week, week_reduced)

ds_labels: unstacked final dataset (lat, lon, week)
***

In [6]:
import xarray as xr
import numpy as np
import pandas as pd

#import pyxpcm
#from pyxpcm.models import pcm

#import cartopy.crs as ccrs
#import cartopy.feature as cfeature
#import matplotlib.pyplot as plt
#import matplotlib.colors as mcolors
#import matplotlib as mpl

#import seaborn as sns
#sns.set_context("paper")
#with_seaborn = True

#from PIL import Image

#import importlib

import Plotter_OR
from Plotter_OR import Plotter_OR 

from BIC_calculation_OR import *

import subprocess

## Model parameters
***

.....

In [7]:
# number of classes
K=7

## Load training dataset
***

__Choose training dataset__

One year of Ocean Color data in the Mediterranean Sea:  OCEANCOLOUR_GLO_CHL_L4_REP_OBSERVATIONS_009_082 product

..........

In [8]:
CMEMS_user = 'agarcia6'
CMEMS_password = '1802910672Aa#'

..............

In [9]:
# geographical extent
geo_extent = [-5, 42, 30, 46] # [min lon, max lon, min lat, max lat]
# time extent
time_extent = ["2019-01-01", "2018-12-31"] # ["min date", "max date"]
# variable to be predict
var_name = 'CHL' # name in dataset
# file name
file_name = 'oceancolour_glo_chl_l4_rep_observations_009_082_2019.nc'

__Load training dataset__

.......

In [10]:
bashCommand = 'python -m motuclient -u ' + CMEMS_user + ' -p ' + CMEMS_password + ' -m "http://my.cmems-du.eu/motu-web/Motu" \
-s OCEANCOLOUR_GLO_CHL_L4_REP_OBSERVATIONS_009_082-TDS -d dataset-oc-glo-bio-multi-l4-chl_interpolated_4km_daily-rep \
-x ' + str(geo_extent[0]) + ' -X ' + str(geo_extent[1]) + ' -y ' + str(geo_extent[2]) + ' -Y ' + str(geo_extent[3]) + \
' -t "' + time_extent[0] + '" -T "' + time_extent[1] + '" -z 0.0 -Z 2500.0 \
-v ' + var_name + ' -o datasets -f ' + file_name
print(bashCommand)
sp = subprocess.call(bashCommand, shell=True)

python -m motuclient -u agarcia6 -p 1802910672Aa# -m "http://my.cmems-du.eu/motu-web/Motu" -s OCEANCOLOUR_GLO_CHL_L4_REP_OBSERVATIONS_009_082-TDS -d dataset-oc-glo-bio-multi-l4-chl_interpolated_4km_daily-rep -x -5 -X 42 -y 30 -Y 46 -t "2019-01-01" -T "2018-12-31" -z 0.0 -Z 2500.0 -v CHL -o datasets -f oceancolour_glo_chl_l4_rep_observations_009_082_2019.nc


In [ ]:
#file_path = 'datasets/dataset-oc-glo-bio-multi-l4-chl_interpolated_4km_daily-rep_1611225376956.nc' #in datarmor
file_path = 'datasets/dataset-oc-glo-bio-multi-l4-chl_interpolated_4km_daily-rep_1610026811620.nc'

Open dataset

........

In [12]:
file_path = 'datasets/' + file_name
ds = xr.open_dataset(file_path)

ds['time'] = ds.indexes['time'].to_datetimeindex()

print(ds)
#ds

<xarray.Dataset>
Dimensions:  (lat: 385, lon: 1128, time: 1)
Coordinates:
  * time     (time) datetime64[ns] 2019-01-01
  * lat      (lat) float32 46.020832 45.979164 45.9375 ... 30.062498 30.020832
  * lon      (lon) float32 -4.9791613 -4.9374948 ... 41.937508 41.97917
Data variables:
    CHL      (time, lat, lon) float32 ...
Attributes:
    comment:                      average
    distribution_statement:       See CMEMS Data License
    site_name:                    GLO
    parameter_code:               CHL
    creation_time:                16:39:54 UTC
    creation_date:                2020-11-05 UTC
    nb_grid_bins:                 37324800
    easternmost_longitude:        180.0
    references:                   http://www.globcolour.info GlobColour has b...
    registration:                 5
    stop_date:                    2020-07-01 UTC
    lon_step:                     0.041666668
    cmems_product_id:             OCEANCOLOUR_GLO_CHL_L4_REP_OBSERVATIONS_009...
    naming_a

__Quick plot__

In [ ]:
ds[var_name].isel(time=11).plot();

In [ ]:
bins = np.arange(int(ds[var_name].min().values), int(ds[var_name].max().values))
ds[var_name].plot.hist(bins=bins);

## Preprocessing
***

#### __1) Weekly mean for each pixel__

In [ ]:
X = ds.groupby("time.week").mean()
#ds = ds.groupby("time.month").mean()
print(ds)
#ds

Plot histogram in time

In [ ]:
bins = np.arange(int(X[var_name].min().values), int(X[var_name].max().values),0.5)
#bins = np.arange(0, 0.3, 0.002)
histo_2d = [] 
for iweek in range(52):
#for iweek in range(12):
    hist_values, bin_edges = np.histogram(X[var_name].isel(week=iweek).values, bins=bins)
    #hist_values, bin_edges = np.histogram(ds[var_name].isel(month=iweek).values, bins=bins)
    histo_2d.append(hist_values)
    
fig, ax = plt.subplots(figsize=(12,10))

#plt.pcolormesh(bins, ds.month.values, histo_2d, cmap='Reds', edgecolors='black')
plt.pcolormesh(bins, X.week.values, histo_2d, cmap='Reds', edgecolors='black')
cbar = plt.colorbar()
ax.set_xlabel('Temperature (K)')
#ax.set_xlabel('Chlorophyll-a concentration (milligram m-3)')
ax.set_ylabel('Weeks')
cbar.ax.set_ylabel('Counts')

#### __2) Reduce lat lon dimensions to sampling dimension__

In [ ]:
sampling_dims = list(X.dims)
sampling_dims.remove('week')
#sampling_dims.remove('time')
sampling_dims

In [ ]:
X = X.stack({'sampling': sampling_dims})
X = X.rename_dims({'week': 'feature'})
X = X.rename({'week': 'feature'})
#X = X.rename_dims({'time': 'feature'})
#X = X.rename({'time': 'feature'})
print(X)
#X

In [ ]:
X[var_name].plot.hist(bins=bins);

#### __3) Delate all NaN time series using mask__

__Chlorophyll mask__: Create mask from another dataset

In [ ]:
mask_file = 'datasets/med00-ogs-bio-an-fc-d_1611233647884.nc'
mask = xr.open_dataset(mask_file)
mask = mask.squeeze()
mask = mask.rename_dims({'latitude': 'lat', 'longitude':'lon'})
mask = mask.rename({'latitude': 'lat', 'longitude':'lon'})
mask = mask.interp_like(ds)
mask = mask['nppv'].notnull()
#print(ds)
mask.plot()

In [ ]:
stacked_mask = mask.stack({'sampling': sampling_dims})
print(stacked_mask)
#stacked_mask

Apply mask

In [ ]:
X = X[var_name].where(stacked_mask == True, drop=True).to_dataset()
print(X)
#X

Delate time series that are all NaN, not taken in to acount in the mask

In [ ]:
X = X[var_name].where(~X[var_name].isnull(),drop=True).to_dataset()

Recover the dataset (unravel)

In [ ]:
X_unstacked = X[var_name].unstack('sampling').to_dataset(name = var_name)
X_unstacked = X_unstacked.sortby(['lat','lon'])
print(np.shape(X_unstacked[var_name]))
# same lat and lon values in mask and in results
mask = stacked_mask.unstack()
X_unstacked = X_unstacked.reindex_like(mask)
print(np.shape(X_unstacked[var_name]))
print(X_unstacked) 

In [ ]:
X_unstacked[var_name].isel(feature=11).plot();

__There is any NaN in the dataset?__

In [ ]:
#np.any(np.isnan(X[var_name].values))
np.sum(np.isnan(X[var_name].values))

#np.argwhere(np.isnan(X[var_name].values))

#### __4) Interpolation__

Not necessary if using mask created from dataset

In [ ]:
#X = X[var_name].interpolate_na(dim = 'feature', method="linear", fill_value="extrapolate").to_dataset(name = var_name)
X = X[var_name].interpolate_na(dim = 'feature', method="linear", fill_value="extrapolate").to_dataset(name = var_name)
print(X)
#X

__There is any NaN in the dataset?__

In [ ]:
#np.any(np.isnan(X[var_name].values))
np.sum(np.isnan(X[var_name].values))

#### __5) Scaler__

choose the best scaler: https://datascience.stackexchange.com/questions/45900/when-to-use-standard-scaler-and-when-normalizer

__Check dimensions order__
***

In [ ]:
np.shape(X[var_name])

Transpose dataset if needed (sampling x features)

In [ ]:
X = X.transpose("sampling", "feature")
np.shape(X[var_name].values)

***

Apply sklearn __StandardScaler__: 
Standardize features by removing the mean and scaling to unit variance
The standard score of a sample x is calculated as:

    z = (x - u) / s
    
where u is the mean of the training samples or zero if `with_mean=False`, and s is the standard deviation of the training samples or one if `with_std=False`.
Centering and scaling happen independently on each feature by computing the relevant statistics on the samples in the training set. 

shape (n_samples, n_features), calculate mean for each feature

In [ ]:
from sklearn.preprocessing import StandardScaler
X_scale = StandardScaler().fit_transform(X[var_name])
X = X.assign(variables={var_name + "_scaled":(('sampling', 'feature'), X_scale)})
#X = X.assign(variables={var_name + "_scaled":(('feature', 'sampling'), X_scale)})
print(X)
#X

In [ ]:
print(X_scale.min())
print(X_scale.max())

In [ ]:
X[var_name].plot.hist(bins=bins);

__Normalizer__ : shape (n_samples, n_features), calculate mean for each sample

In [ ]:
#from sklearn.preprocessing import Normalizer
#X_scale = Normalizer().fit_transform(X[var_name])
#X = X.assign(variables={var_name + "_scaled":(('sampling', 'feature'), X_scale)})

In [ ]:
#print(X_scale.min())
#print(X_scale.max())

In [ ]:
#X[var_name].plot.hist(bins=bins);

__MinMaxScaler__: scale each feature. shape (n_samples, n_features)

In [ ]:
#from sklearn.preprocessing import MinMaxScaler
#X_scale = MinMaxScaler().fit_transform(X[var_name])
#X = X.assign(variables={var_name + "_scaled":(('sampling', 'feature'), X_scale)})

In [ ]:
#print(X_scale.min())
#print(X_scale.max())

In [ ]:
#print(X[var_name].min().values)
#print(X[var_name].max().values)

In [ ]:
#X[var_name].plot.hist(bins=bins);

#### __5) PCA__

Apply __Principal component analysis__ (PCA):
Linear dimensionality reduction using Singular Value Decomposition of the data to project it to a lower dimensional space.
If `0 < n_components < 1` and `svd_solver == 'full'`, select the number of components such that the amount of variance that needs to be explained is greater than the percentage specified by n_components.

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 0.99, svd_solver = 'full')
pca

In [ ]:
pca = pca.fit(X[var_name + "_scaled"])

In [ ]:
fig, ax = plt.subplots()
pb = plt.bar(range(pca.n_components_), pca.explained_variance_ratio_)
ax.set_xlabel('n_components')
ax.set_ylabel('Percentage')
ax.set_title('Percentage of variance explained by each of the selected components')

In [ ]:
X_reduced = pca.transform(X[var_name + "_scaled"])
np.shape(X_reduced)

In [ ]:
X = X.assign(variables={var_name + "_reduced":(('sampling', 'feature_reduced'),X_reduced)})
print(X)

## Apply Model
***

__Create model__

In [ ]:
from sklearn import mixture
k = 7 # number of classes
model = mixture.GaussianMixture(n_components=k, covariance_type='full')
model

In [ ]:
#from sklearn.cluster import KMeans
#k=7
#model = KMeans(n_clusters=7)
#model

__Fit model__

In [ ]:
model.fit(X[var_name + "_reduced"])

__Predict labels__

In [ ]:
X_labels = model.predict(X[var_name + "_reduced"])
X_labels

In [ ]:
print(X_labels.min())
print(X_labels.max())

In [ ]:
X = X.assign(variables={"GMM_labels":(('sampling'),X_labels)})
print(X)
#X

Calculate other __stadistics__

- Predict posterior probability of each component given the data

In [ ]:
X_proba = model.predict_proba(X[var_name + "_reduced"])
np.shape(X_proba)

In [ ]:
X = X.assign(variables={"GMM_post":(('sampling','k'),X_proba)})
print(X)
#X

- Calculate quantiles

In [ ]:
#quantiles we want to calculate
q = [0.05, 0.5, 0.95]

In [ ]:
m_quantiles = X[var_name].where(X['GMM_labels']==0, drop=True).quantile(q, dim='sampling')
for yi in range(1,k):
    m_quantiles = xr.concat((m_quantiles, X[var_name].where(X['GMM_labels']==yi, drop=True).quantile(q, dim='sampling')), dim='k')

In [ ]:
X = X.assign(variables={var_name + "_Q":(('k','quantile','feature'), m_quantiles)})
X = X.assign_coords(coords={'quantile': q})
print(X)

- Calculate quantiles scaled variable

In [ ]:
#quantiles we want to calculate
q = [0.05, 0.5, 0.95]

In [ ]:
m_quantiles = X[var_name].where(X['GMM_labels']==0, drop=True).quantile(q, dim='sampling')
for yi in range(1,k):
    m_quantiles = xr.concat((m_quantiles, X[var_name + '_scaled'].where(X['GMM_labels']==yi, drop=True).quantile(q, dim='sampling')), dim='k')

In [ ]:
X = X.assign(variables={var_name + '_scaled'+ "_Q":(('k','quantile','feature'), m_quantiles)})
X = X.assign_coords(coords={'quantile': q})
print(X)

- Calculate robustness

In [ ]:
maxpost = X["GMM_post"].max(dim="k")
K = len(X["GMM_labels"])
robust = (maxpost - 1. / K) * K / (K - 1.)

Plist = [0, 0.33, 0.66, 0.9, .99, 1]
rowl0 = ('Unlikely', 'As likely as not', 'Likely', 'Very Likely', 'Virtually certain')
robust_id = np.digitize(robust, Plist) - 1

In [ ]:
X = X.assign(variables={"GMM_robustness":(('sampling'), robust), "GMM_robustness_cat":(('sampling'), robust_id)})
X["GMM_robustness_cat"].attrs['legend'] = rowl0
print(X)
#X

__Unstack dataset__

In [ ]:
ds_labels = X.unstack('sampling')
#ds_labels = ds_labels.sortby(['lat','lon'])
# same lat and lon values in mask and in results
mask = stacked_mask.unstack()
ds_labels = ds_labels.reindex_like(mask)
print(ds_labels)
#ds_labels

In [ ]:
#copy atributtes
ds_labels.attrs = ds.attrs
ds_labels.lat.attrs = ds.lat.attrs
ds_labels.lon.attrs = ds.lon.attrs
#include time coord for save_BlueCloud function
ds_labels = ds_labels.assign_coords({'time': ds.time.values})
ds_labels

In [ ]:
ds_labels['CHL_reduced'].coords

## Preprocesing plots
***

In [ ]:
P = Plotter_OR(ds_labels, model, coords_dict={'latitude':'lat', 'longitude':'lon', 'feature': 'feature'})

__Scatter plot__

In [ ]:
P.scatter_PDF(var_name = var_name + '_reduced')
P.save_BlueCloud('figures/scatter_PDF_EX_chl.png')

__BIC__

In [ ]:
corr_dist = 50 # correlation distance in km
Nrun = 10 # number of runs for each k
NK = 20 # max number of classes to explore

In [ ]:
BIC, BIC_min = BIC_calculation(X=X, coords_dict={'latitude':'lat', 'longitude':'lon'}, 
                               corr_dist=corr_dist,
                               feature_name='feature_reduced', var_name= 'CHL_reduced',
                               Nrun=Nrun, NK=NK)

In [ ]:
plot_BIC(BIC, NK=NK)
P.save_BlueCloud('figures/BIC_EX_chl.png', bic_fig='yes')

## Plot results
***

In [ ]:
P = Plotter_OR(ds_labels, model, coords_dict={'latitude':'lat', 'longitude':'lon', 'feature': 'feature'})

#### __1) Quantiles time series__

Median and other quantiles representation

In [ ]:
P.tseries_structure(q_variable = var_name + '_Q', start_month=6, ylabel='Temperature (K)')
P.save_BlueCloud('figures/tseries_struc_EX_chl.png')

All median time series in the same plot 

In [ ]:
P.tseries_structure_comp(q_variable = var_name + '_Q', plot_q= 'all', ylabel='Temperature (K)', start_month=6)
P.save_BlueCloud('figures/tseries_struc_comp_EX_chl.png')

If temperature is normalized (__how we can interpret this figure?__)

In [ ]:
P.tseries_structure(q_variable = var_name + '_scaled' + '_Q', start_month=6, ylabel='Temperature (K)')
P.save_BlueCloud('figures/tseries_struc_EX_norm_chl.png')

#### __2) Spatial distribution of classes__

In [ ]:
P.spatial_distribution()
P.save_BlueCloud('figures/spatial_distr_EX_chl.png')

#### __3) Robustness__

In [ ]:
P.plot_robustness()
P.save_BlueCloud('figures/robustness_EX_chl.png')

#### __4) Classes pie chart__

In [ ]:
P.pie_classes()
P.save_BlueCloud('figures/pie_chart_EX_chl.png')